# Day 02 — Prompt chaining (planner → executor → critic)

This notebook builds a tiny chain: a planner creates steps, executors fill them in, and a critic reviews the final answer.

In [ ]:
import asyncio
from pathlib import Path

from openai import OpenAI

client = OpenAI()
PROMPTS_DIR = Path("prompts")

def load_prompt(name: str) -> str:
    return (PROMPTS_DIR / name).read_text()

def render(template: str, **kwargs: str) -> str:
    text = template
    for key, value in kwargs.items():
        text = text.replace(f"{{{key}}}", value)
    return text

def call_openai(prompt: str, model: str = "gpt-4o-mini") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content.strip()

async def call_openai_async(prompt: str, model: str = "gpt-4o-mini") -> str:
    return await asyncio.to_thread(call_openai, prompt, model)


In [ ]:
request = "Design a one-week plan to learn the basics of neural networks."

planner_prompt = render(
    load_prompt("planner.txt"),
    request=request,
)

plan = call_openai(planner_prompt)
plan

## Sequential execution
Each step is completed one at a time. This makes it easier to keep context, but it can be slower.

In [ ]:
steps = [line for line in plan.splitlines() if line.strip()]

executor_template = load_prompt("executor.txt")

sequential_results = []
for step in steps:
    prompt = render(executor_template, context=request, step=step)
    sequential_results.append(call_openai(prompt))

sequential_results

## Parallel execution
If steps are independent, they can run concurrently to save time.

In [ ]:
async def run_parallel(steps_list: list[str]) -> list[str]:
    tasks = []
    for step in steps_list:
        prompt = render(executor_template, context=request, step=step)
        tasks.append(asyncio.create_task(call_openai_async(prompt)))
    return await asyncio.gather(*tasks)

parallel_results = await run_parallel(steps)
parallel_results

## Critic review
A quick sanity check on the combined answer.

In [ ]:
combined_answer = "\n".join(parallel_results)
critic_prompt = render(load_prompt("critic.txt"), answer=combined_answer)
critic_feedback = call_openai(critic_prompt)
critic_feedback